## Get GPU on computer

In [1]:
import keras

Using TensorFlow backend.


In [2]:
import torch
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


## Extracting Data

In [3]:
import pandas as pd
import os
import random

In [4]:
def _read_wiki(data_dir,file):
    file_name = os.path.join(data_dir, file)
    with open(file_name, 'r',encoding="utf8") as f:
        lines = f.readlines()
    # Uppercase letters are converted to lowercase ones
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

In [5]:
paragraphs_train = _read_wiki(r'C:\Users\lpott\Desktop\Coursera\Projects\PyTorch_BERT_Tutorial','valid.txt')
paragraphs_test = _read_wiki(r'C:\Users\lpott\Desktop\Coursera\Projects\PyTorch_BERT_Tutorial','test.txt')

In [6]:
print(paragraphs_train[0])

['a small number of australian volunteers fought on both sides of the spanish civil war , although they predominantly supported the spanish republic through the international brigades', 'the australians were subsequently allocated to the battalions of other <unk> , such as the british battalion and the lincoln battalion , rather than forming their own units', 'most were radicals motivated by ideological reasons , while a number were spanish @-@ born migrants who returned to fight in their country of origin', "at least 66 australians volunteered , with only one — <unk> bull , a conservative <unk> who was later killed serving in the raf during the second world war — known to have fought for general francisco franco 's nationalist forces ."]


In [7]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = 1
    else:
        # `paragraphs` is a list of lists of lists
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = 0
    return sentence, next_sentence, is_next

In [8]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        sent_a, sent_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        
        nsp_data_from_paragraph.append((sent_a, sent_b, is_next))
    return nsp_data_from_paragraph

In [9]:
_get_next_sentence(paragraphs_train[0][0],paragraphs_train[0][1],paragraphs_train)

('a small number of australian volunteers fought on both sides of the spanish civil war , although they predominantly supported the spanish republic through the international brigades',
 'the australians were subsequently allocated to the battalions of other <unk> , such as the british battalion and the lincoln battalion , rather than forming their own units',
 1)

In [10]:
_get_nsp_data_from_paragraph(paragraphs_train[0],paragraphs_train)

[('a small number of australian volunteers fought on both sides of the spanish civil war , although they predominantly supported the spanish republic through the international brigades',
  'the name fell out of use before its first anniversary because of world war i.',
  0),
 ('the australians were subsequently allocated to the battalions of other <unk> , such as the british battalion and the lincoln battalion , rather than forming their own units',
  'judge <unk> further <unk> blythe for not having met with the <unk> family .',
  0),
 ('most were radicals motivated by ideological reasons , while a number were spanish @-@ born migrants who returned to fight in their country of origin',
  "at least 66 australians volunteered , with only one — <unk> bull , a conservative <unk> who was later killed serving in the raf during the second world war — known to have fought for general francisco franco 's nationalist forces .",
  1)]

In [11]:
examples_train = []
for paragraph in paragraphs_train:
    examples_train.extend(_get_nsp_data_from_paragraph(
        paragraph, paragraphs_train))

In [12]:
examples_test = []
for paragraph in paragraphs_test:
    examples_test.extend(_get_nsp_data_from_paragraph(
        paragraph, paragraphs_test))

In [13]:
sent_a_train,sent_b_train,isnext_train = zip(*examples_train)

In [14]:
sent_a_test,sent_b_test,isnext_test= zip(*examples_test)

## Tokenization, padding, truncation,attention masks and Input Formatting

In [15]:
from transformers import BertTokenizer
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [16]:
encoded_dict_train = tokenizer(sent_a_train, sent_b_train,padding='max_length',
                         truncation=True,max_length=512)
encoded_dict_test = tokenizer(sent_a_test, sent_b_test,padding='max_length',
                         truncation=True,max_length=512)

In [ ]:
encoded_dict.keys()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-cbf1662c1931>", line 1, in <module>
    encoded_dict.keys()
NameError: name 'encoded_dict' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\lpott\anacon

## Convert to tensor

In [17]:
train_inputs = torch.tensor(encoded_dict_train['input_ids'])

In [18]:
train_inputs.shape

ERROR! Session/line number was not unique in database. History logging moved to new session 799


torch.Size([6216, 512])

In [19]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(encoded_dict_train['input_ids'])
validation_inputs = torch.tensor(encoded_dict_test['input_ids'])
train_labels = torch.tensor(isnext_train)
validation_labels = torch.tensor(isnext_test)
train_masks = torch.tensor(encoded_dict_train['attention_mask'])
validation_masks = torch.tensor(encoded_dict_test['attention_mask'])
train_segment_id = torch.tensor(encoded_dict_train['token_type_ids'])
validation_segment_id = torch.tensor(encoded_dict_test['token_type_ids'])

In [20]:
validation_segment_id.shape

torch.Size([7182, 512])

## Create training iterator

In [21]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.
batch_size = 2
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_segment_id, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_segment_id,validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## Fine Tune Classification Model

In [22]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
# Tell pytorch to run this model on the GPU.
model.cuda()

- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Get all model layers

In [23]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

## Optimizer

In [24]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 10
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

## Train loop

In [25]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [26]:
import time
import datetime
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [27]:
print("Running Validation...")
t0 = time.time()
# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()
# Tracking variables 
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
# Evaluate data for one epoch
for batch in validation_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_token_ids, b_labels = batch

    # Telling the model not to compute or store gradients, saving memory and
    # speeding up validation
    with torch.no_grad():        
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have
        # not provided labels.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids.type(torch.LongTensor).to(device), 
                        token_type_ids=b_token_ids.to(device), 
                        attention_mask=b_input_mask.type(torch.LongTensor).to(device))

    # Get the "logits" output by the model. The "logits" are the output
    # values prior to applying an activation function like the softmax.
    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy
    # Track the number of batches
    nb_eval_steps += 1
# Report the final accuracy for this validation run.
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

Running Validation...
  Accuracy: 0.32
  Validation took: 0:03:25


In [ ]:
import random
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    torch.cuda.empty_cache()
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].type(torch.LongTensor).to(device)
        b_input_mask = batch[1].to(device)
        b_token_ids = batch[2].to(device)
        b_labels = batch[3].type(torch.LongTensor).to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=b_token_ids, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_token_ids, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids.type(torch.LongTensor).to(device), 
                            token_type_ids=b_token_ids.to(device), 
                            attention_mask=b_input_mask.type(torch.LongTensor).to(device))
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...
  Batch    40  of  3,108.    Elapsed: 0:00:10.
  Batch    80  of  3,108.    Elapsed: 0:00:19.
  Batch   120  of  3,108.    Elapsed: 0:00:29.
  Batch   160  of  3,108.    Elapsed: 0:00:38.
  Batch   200  of  3,108.    Elapsed: 0:00:47.
  Batch   240  of  3,108.    Elapsed: 0:00:57.
  Batch   280  of  3,108.    Elapsed: 0:01:06.
  Batch   320  of  3,108.    Elapsed: 0:01:15.
  Batch   360  of  3,108.    Elapsed: 0:01:25.
  Batch   400  of  3,108.    Elapsed: 0:01:34.
  Batch   440  of  3,108.    Elapsed: 0:01:43.
  Batch   480  of  3,108.    Elapsed: 0:01:53.
  Batch   520  of  3,108.    Elapsed: 0:02:02.
  Batch   560  of  3,108.    Elapsed: 0:02:11.
  Batch   600  of  3,108.    Elapsed: 0:02:20.
  Batch   640  of  3,108.    Elapsed: 0:02:30.
  Batch   680  of  3,108.    Elapsed: 0:02:39.
  Batch   720  of  3,108.    Elapsed: 0:02:48.
  Batch   760  of  3,108.    Elapsed: 0:02:57.
  Batch   800  of  3,108.    Elapsed: 0:03:07.
  Batch   840  o

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-a9d3e556608a>", line 82, in <module>
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\torch\nn\utils\clip_grad.py", line 33, in clip_grad_norm_
    total_norm += param_norm.item() ** norm_type
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\e

In [28]:
encode = tokenizer("The man went on a long journey.","but he never was able to finish the quest.")

In [29]:
encode.keys()

ERROR! Session/line number was not unique in database. History logging moved to new session 800


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [30]:
in_ids = torch.tensor(encode['input_ids']).unsqueeze(0)
seg_ids =  torch.tensor(encode['token_type_ids']).unsqueeze(0)
att =  torch.tensor(encode['attention_mask']).unsqueeze(0)

In [31]:
with torch.no_grad():        
    # Forward pass, calculate logit predictions.
    # This will return the logits rather than the loss because we have
    # not provided labels.
    # token_type_ids is the same as the "segment ids", which 
    # differentiates sentence 1 and 2 in 2-sentence tasks.
    # The documentation for this `model` function is here: 
    # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    outputs = model(in_ids.type(torch.LongTensor).to(device), 
                    token_type_ids=seg_ids.to(device), 
                    attention_mask=att.type(torch.LongTensor).to(device))

In [32]:
outputs[0].argmax()

tensor(1, device='cuda:0')